In [47]:
import pandas as pd
import numpy as np
import math as m
from IPython.display import display, HTML

def isNegative(num):
    return  num < 0



def biseccion(threshold, f, epsilon, numIterations):
    x0 = threshold[0]
    x1 = threshold[1]
    assert (isNegative(f.evalf(subs={x:x0})) != isNegative(f.evalf(subs={x:x1}))),"Biseccion: Los signos de los limites deben ser diferentes"
    
    values = pd.DataFrame(columns=('x0', 'x1', 'xp', 'f(x0)', 'f(x1)', 'f(xp)', 'error'))
    i = 0
    error = m.fabs(x0 - x1)
    while((i < numIterations) and (error > epsilon)):
        xp = x0 + (m.fabs(x0-x1)/2)
        f_x0 = f.evalf(subs={x:x0})
        f_x1 = f.evalf(subs={x:x1})
        biseccion = f.evalf(subs={x:xp})
        if(biseccion == 0):
            return values
        elif(isNegative(f_x0) == isNegative(biseccion)):
            x0 = xp
        elif(isNegative(f_x1) == isNegative(biseccion)): 
            x1 = xp
        i = i + 1 
        error = m.fabs(x0 - x1)
        values.loc[len(values)] = [x0, x1, xp, f_x0, f_x1, biseccion, error]
    return values 

def newtonRaphson(point, f, epsilon, numIterations):
    derivative = f.diff(x)
    values = pd.DataFrame(columns=('x', 'new_x', 'f(x)', 'f(x_new_x)', 'error'))
    error = -1
    i = 0
    while((i < numIterations) and (error > epsilon or error == -1)):
        f_point = f.evalf(subs={x:point})
        fp_point = derivative.evalf(subs={x:point})
        point2 = point - (f_point)/(fp_point)
        f_newPoint = f.evalf(subs={x:point2})
        error = m.fabs(f_newPoint)
        values.loc[len(values)] = [point, point2, f_point, f_newPoint, error]
        point = point2
        i = i + 1
    return values

    
    
        

In [50]:
from sympy.abc import x

y = -(0.3 * (x ** 4)) + (1.2 * (x ** 3)) - (1.8 * (x ** 2)) + (4 * x) 

#v = biseccion((3,4), y, 0.0001, 100)
v = newtonRaphson(2, y, 0.0001, 100)
v

,x,new_x,f(x),f(x_new_x),error
0,2,-1.50000000000000,5.60000000000000,-15.6187500000001,1.561875e+01
1,-1.50000000000000,-0.775232018561487,-15.6187500000001,-4.85013836189075,4.850138e+00
2,-0.775232018561487,-0.265413642582549,-4.85013836189075,-1.21237950374803,1.212380e+00
3,-0.265413642582549,-0.0336687026594466,-1.21237950374803,-0.136761442373427,1.367614e-01
4,-0.0336687026594466,-0.000517098056048812,-0.136761442373427,-0.00206887369285639,2.068874e-03
5,-0.000517098056048812,-1.20352616465045e-7,-0.00206887369285639,-4.81410491932735e-7,4.814105e-07


In [40]:


y = -(0.3 * (x ** 4)) + (1.2 * (x ** 3)) - (1.8 * (x ** 2)) + (4 * x) 
y


-0.3*x**4 + 1.2*x**3 - 1.8*x**2 + 4*x